In [1]:
import pynverse
import numpy as np
import sys
import random

sys.path.append("..")

from rascal import models
from rascal import util

import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

import numpy as np

from rascal.calibrator import Calibrator
from rascal.util import refine_peaks


c:\users\josh\.conda\envs\rascal\lib\site-packages\rascal\calibrator.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
class SyntheticSpectrum:
    def __init__(self, coefficients, model_type='cubic', degree=None):
        """
        Creates a synthetic spectrum generator which, given a suitable model,
        outputs the expected pixel locations of input wavelengths.

        Parameters
        ----------
        coefficients:
          list, coefficients for the model
        model_type:
           str, model type (linear, quadratic, cubic or poly)
        degree:
           int, if using a general poly model, its degree, default None

        It is
        expected that this will be used mainly for model testing, but
        you can alsus
        """
        self.model = None

        # Default is approx. range of Silicon
        # Angstrom
        self.min_wavelength = 2000
        self.max_wavelength = 12000
        
        self.coefficients = coefficients

        # Model to fit
        if model_type == 'quadratic':
            self.model = models.quadratic(self.coefficients)
        elif model_type == 'cubic':
            self.model = models.cubic(self.coefficients)
        elif model_type == 'poly':

            if degree is None:
                raise ValueError("You should specify a polynomial degree.")

            self.model = models.polynomial(self.coefficients, degree)
        else:
            raise NotImplementedError

    def set_wavelength_limit(self, min_w, max_w):
        """
        Set a wavelength filter for the `get_pixels` function.
        """
        self.min_wavelength = min_w
        self.max_wavelength = max_w

    def get_pixels(self, wavelengths):
        """
        Returns a list of pixel locations for the wavelengths provided
        """

        if self.model is None:
            raise ValueError("Model not initiated")

        wavelengths = np.array(wavelengths)
        wavelengths = wavelengths[wavelengths >= self.min_wavelength]
        wavelengths = wavelengths[wavelengths <= self.max_wavelength]

        return pynverse.inversefunc(self.model, wavelengths)

class RandomSyntheticSpectrum(SyntheticSpectrum):

    def __init__(self, num_pixels=1024, min_wavelength=4000, max_wavelength=8000, endpoint_jitter=100, model_type='poly', degree=1):

        # This is the maximum and minimum wavelength of the sensor
        min_wavelength = min_wavelength +endpoint_jitter*(-1 + 2*random.random())
        max_wavelength += endpoint_jitter*(-1 + 2*random.random())
        self.num_pixels = 1024
        
        x0 = min_wavelength
        x1 = (max_wavelength - x0)/num_pixels
        
        coefficients = [x0, x1]
        
        if degree >= 2:
            coefficients.append(0.002*random.random())
            
        if degree >= 3:
            coefficients.append(-0.001+0.002*random.random())

        coefficients.reverse()

        super().__init__(coefficients, model_type, degree)
        self.set_wavelength_limit(x0, max_wavelength)

    def add_atlas(self, elements, n_lines=30, min_intensity=1000, min_distance=10):
        lines = util.load_calibration_lines(elements,
                           min_atlas_wavelength=self.min_wavelength,
                           max_atlas_wavelength=self.max_wavelength,
                           min_intensity=min_intensity,
                           min_distance=min_distance,
                           vacuum=False)
    
        idx = sorted(random.sample(range(len(lines[0])), min(len(lines[0]), n_lines)))
        
        self.elements = lines[0][idx]
        self.wavelengths = lines[1][idx]
        self.intensities = lines[2][idx]

In [3]:
def test_fit(spectrum, uncertainty = 0, range_tolerance=500, elements=['Hg', 'Ar']):
    c = Calibrator(pixs)
               
    c.set_calibrator_properties(num_pix=spectrum.num_pixels, plotting_library='matplotlib',
                                log_level='info') 

    c.set_hough_properties(num_slopes=5000,
                           xbins=100,
                           ybins=100,
                           min_wavelength=spectrum.min_wavelength,
                           max_wavelength=spectrum.max_wavelength,
                           range_tolerance=range_tolerance,
                           linearity_tolerance=100)

    #c.add_atlas(elements, min_atlas_wavelength=spectrum.min_wavelength - uncertainty,
    #                        max_atlas_wavelength=spectrum.max_wavelength + uncertainty)
    
    c.add_user_atlas(['Hg']*len(spectrum.wavelengths), spectrum.wavelengths)
    
    print(c.atlas)

    c.set_ransac_properties(sample_size=5,
                            top_n_candidate=5,
                            linear=True,
                            filter_close=True,
                            ransac_tolerance=5,
                            candidate_weighted=True,
                            hough_weight=1.0)

    c.do_hough_transform()
    
    return c

In [6]:
"""
Spectrum setup
"""
min_wavelength = 4000
max_wavelength = 8000
endpoint_jitter = 0
elements = ['Hg', 'Ar']
poly_degree = 1
num_pixels = 1024

"""
Generate spectrum + ground truth
"""
spectrum = RandomSyntheticSpectrum(num_pixels,
                                   min_wavelength,
                                   max_wavelength,
                                   endpoint_jitter,
                                   'poly',
                                   poly_degree)
spectrum.add_atlas(elements)

wavs = spectrum.wavelengths
pixs = spectrum.get_pixels(spectrum.wavelengths)

ground_truth = zip(wavs, pixs)

print(list(ground_truth))

calibrator = test_fit(spectrum, uncertainty=0, range_tolerance=400, elements=elements)
polyfit_coeff, rms, residual, peak_utilisation = calibrator.fit(fit_deg=1, max_tries=500)

print(polyfit_coeff, residual, spectrum.coefficients)

"""
Check fit
"""
fit_model = models.polynomial(polyfit_coeff[::-1], 1)
errs = np.array([fit_model(px) for px in pixs]) - wavs

print(list(zip(errs, wavs)))

INFO:rascal.calibrator:num_pix is set to None.
INFO:rascal.calibrator:pixel_list is set to None.
INFO:rascal.calibrator:Plotting with matplotlib.
INFO:rascal.calibrator:num_pix is set to 1024.
INFO:rascal.calibrator:pixel_list is set to None.
INFO:rascal.calibrator:Plotting with matplotlib.


[(4046.544, 11.91524999999928), (4077.8152, 19.920687499997815), (4358.306, 91.72637499994562), (5460.697, 373.9383750000115), (5769.5503, 453.00487500001043), (6907.3833, 744.2901249999412), (6965.3486, 759.1292500001169), (7067.1343, 785.1863750000688), (7081.817, 788.9451250001406), (7146.9565, 805.6208750000194), (7272.8477, 837.8490000001486), (7383.89, 866.2758749999145), (7503.7764, 896.9667499999335), (7514.5596, 899.7272499999634), (7635.0117, 930.5630000001621), (7948.0757, 1010.7073750001598)]
[4046.544, 4077.8152, 4358.306, 5460.697, 5769.5503, 6907.3833, 6965.3486, 7067.1343, 7081.817, 7146.9565, 7272.8477, 7383.89, 7503.7764, 7514.5596, 7635.0117, 7948.0757]



[4.00000e+03 3.90625e+00] [-2.84671842e-09 -2.84444468e-09 -2.97404767e-09 -2.42653186e-09
 -2.34922481e-09 -2.31921149e-09 -1.61799107e-09 -1.49520929e-09
 -1.95177563e-09 -1.41335477e-09 -2.29829311e-09 -2.07364792e-09
 -1.26510713e-09 -1.19143806e-09] [3.90625, 4000.0]
[(-2.8467184165492654e-09, 4046.544), (-2.844444679794833e-09, 4077.8152), (-2.9740476747974753e-09, 4358.306), (-2.4265318643301725e-09, 5460.697), (-2.3492248146794736e-09, 5769.5503), (-2.319211489520967e-09, 6907.3833), (-1.6179910744540393e-09, 6965.3486), (-1.7789716366678476e-09, 7067.1343), (-1.495209289714694e-09, 7081.817), (-1.951775630004704e-09, 7146.9565), (-1.4133547665551305e-09, 7272.8477), (-2.2982931113801897e-09, 7383.89), (-2.191882231272757e-09, 7503.7764), (-2.0736479200422764e-09, 7514.5596), (-1.2651071301661432e-09, 7635.0117), (-1.191438059322536e-09, 7948.0757)]


In [5]:
candidates = np.array(list(zip(calibrator.candidate_peak, calibrator.candidate_arc)))

for gt in list(zip(pixs, wavs)):
    peak, arc = gt

    print("-- Checking ground truth arc: {} --".format(arc))

    atlas_idx = np.argwhere(np.abs(calibrator.atlas - arc) < 1e-5)
    if len(atlas_idx) == 0:
        print("\tLine is not in atlas.")
    else:
        idxs = np.argwhere(np.abs(candidates[:,1] - arc) < 1e-5)

        if len(idxs) == 0:
            print("\tIn atlas, but not a candidate")
        else:
            print("\tCandidate found")

-- Checking ground truth arc: 4046.5439453125 --
	Candidate found
-- Checking ground truth arc: 4077.815185546875 --
	Candidate found
-- Checking ground truth arc: 4358.30615234375 --
	Candidate found
-- Checking ground truth arc: 5460.69677734375 --
	Candidate found
-- Checking ground truth arc: 5769.55029296875 --
	Candidate found
-- Checking ground truth arc: 6907.38330078125 --
	Candidate found
-- Checking ground truth arc: 6965.3486328125 --
	Candidate found
-- Checking ground truth arc: 7067.13427734375 --
	Candidate found
-- Checking ground truth arc: 7081.81689453125 --
	Candidate found
-- Checking ground truth arc: 7146.95654296875 --
	Candidate found
-- Checking ground truth arc: 7272.84765625 --
	Candidate found
-- Checking ground truth arc: 7383.89013671875 --
	Candidate found
-- Checking ground truth arc: 7503.7763671875 --
	Candidate found
-- Checking ground truth arc: 7514.5595703125 --
	Candidate found
-- Checking ground truth arc: 7635.01171875 --
	Candidate found
-- C